In [1]:
import torch
from torch.autograd import Variable
import utils
import dataset
from PIL import Image

import models.crnn as crnn
import params
import argparse
from IPython.core import display
import glob


In [2]:

model_path = '/home/niddal/Desktop/PhD_projects/Arabic-text-recognition-master/expr/l-4/netCRNN_5_39000-83.pth'

# net init
nclass = len(params.alphabet) + 1
model = crnn.CRNN(params.imgH, params.nc, nclass, params.nh)
# load model
print('loading pretrained model from %s' % model_path)

model.load_state_dict(torch.load(model_path))

converter = utils.strLabelConverter(params.alphabet)

transformer = dataset.resizeNormalize((100, 32))


loading pretrained model from /home/niddal/Desktop/PhD_projects/Arabic-text-recognition-master/expr/l-4/netCRNN_5_39000-83.pth


/usr/lib/python3/dist-packages/torch/nn/modules/rnn.py:54: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


RuntimeError: CUDA error: all CUDA-capable devices are busy or unavailable

In [50]:
#recognize and save results in a text file
for img_path in glob.glob('/home/niddal/Desktop/PhD_projects/Arabic-text-recognition-master/data/osn_100/train_images_100/*.jpg'):
    image = Image.open(img_path).convert('L')
    image = transformer(image)
    image = image.view(1, *image.size())
    image = Variable(image)

    model.eval()
    preds = model(image)

    _, preds = preds.max(2)
    preds = preds.transpose(1, 0).contiguous().view(-1)
    
    with open('/home/niddal/Desktop/PhD_projects/Arabic-text-recognition-master/output.txt','a') as fp:

        preds_size = Variable(torch.IntTensor([preds.size(0)]))
        raw_pred = converter.decode(preds.data, preds_size.data, raw=True)
        sim_pred = converter.decode(preds.data, preds_size.data, raw=False)
        #display.display(display.Image(image_path))
        #print('%-20s => %-20s' % (raw_pred, sim_pred))
        fp.write(sim_pred)
        fp.write(image_path)
        fp.write('\n')
    fp.close()



In [3]:
#Display test image and recognize it
model_path = '/home/niddal/Desktop/PhD_projects/Arabic-text-recognition-master/expr/l-4/netCRNN_5_39000-83.pth'
#image_path = '/home/niddal/Desktop/PhD_projects/Arabic-text-recognition-master/data/osn_100/train_images_100/ae_02.jpg'
image_path = '/home/niddal/Desktop/PhD_projects/Data/ch8_test_word_images/word_1.png'

# net init
nclass = len(params.alphabet) + 1
model = crnn.CRNN(params.imgH, params.nc, nclass, params.nh)
if torch.cuda.is_available():
    model = model.cuda()

# load model
print('loading pretrained model from %s' % model_path)
if params.multi_gpu:
    model = torch.nn.DataParallel(model)
model.load_state_dict(torch.load(model_path))

converter = utils.strLabelConverter(params.alphabet)

transformer = dataset.resizeNormalize((100, 32))
image = Image.open(image_path).convert('L')
image = transformer(image)
if torch.cuda.is_available():
    image = image.cuda()
image = image.view(1, *image.size())
image = Variable(image)

model.eval()
preds = model(image)

_, preds = preds.max(2)
preds = preds.transpose(1, 0).contiguous().view(-1)

preds_size = Variable(torch.IntTensor([preds.size(0)]))
raw_pred = converter.decode(preds.data, preds_size.data, raw=True)
sim_pred = converter.decode(preds.data, preds_size.data, raw=False)
display.display(display.Image(image_path))
print(image_path)
print(image_path, sim_pred)

print('%-20s => %-20s' % (raw_pred, sim_pred))



/usr/lib/python3/dist-packages/torch/nn/modules/rnn.py:54: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


RuntimeError: CUDA error: all CUDA-capable devices are busy or unavailable